In [16]:
import pandas as pd
import string
import nltk
nltk.download('stopwords')
from collections import Counter
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aadit\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [40]:
stopwords_dict = Counter(stopwords.words('english'))

In [41]:
def docPreProcessing(filepath):
    file = open(filepath, 'r')
    lines = file.readlines()
    
    
    doctext = ""
    for line in lines:
        line = line.translate(str.maketrans('', '', string.punctuation)).strip().lower()
        line = ' '.join([word for word in line.split() if word not in stopwords_dict])
        doctext += line
        
    doctextList = doctext.split()
    return doctextList
        


In [54]:
def tf(wordlist):
    wordmap = {}
    
    for word in wordlist:
        if word in wordmap:
            wordmap[word] += 1
        else:
            wordmap[word] = 1
            
    
    numWords = len(wordmap)
    print(numWords)
    
    for word in wordmap:
        wordmap[word] = wordmap[word] / numWords
    
    return wordmap
        


In [55]:
tf(docPreProcessing('sample1.txt'))

122


{'birds': 0.09836065573770492,
 'fascinating': 0.01639344262295082,
 'creatures': 0.01639344262295082,
 'planet': 0.00819672131147541,
 'come': 0.00819672131147541,
 'variety': 0.01639344262295082,
 'shapes': 0.00819672131147541,
 'sizes': 0.00819672131147541,
 'colors': 0.00819672131147541,
 'behavior': 0.01639344262295082,
 'entertaining': 0.00819672131147541,
 'aweinspiring': 0.00819672131147541,
 'tiny': 0.00819672131147541,
 'hummingbird': 0.00819672131147541,
 'majestic': 0.01639344262295082,
 'eagle': 0.01639344262295082,
 'constant': 0.00819672131147541,
 'presence': 0.00819672131147541,
 'lives': 0.00819672131147541,
 'whether': 0.02459016393442623,
 'realizeone': 0.00819672131147541,
 'interesting': 0.01639344262295082,
 'things': 0.00819672131147541,
 'ability': 0.00819672131147541,
 'fly': 0.00819672131147541,
 'better': 0.00819672131147541,
 'others': 0.01639344262295082,
 'wings': 0.01639344262295082,
 'allow': 0.00819672131147541,
 'soar': 0.00819672131147541,
 'air': 0.